Lettura DataSet

In [19]:
import pandas as pd 

df_classes = pd.read_csv("elliptic_txs_classes.csv")
df_edgelist = pd.read_csv("elliptic_txs_edgelist.csv")
df_features = pd.read_csv("elliptic_txs_features.csv",header=None)


              0    1         2         3         4          5         6    \
0       230425980    1 -0.171469 -0.184668 -1.201369  -0.121970 -0.043875   
1         5530458    1 -0.171484 -0.184668 -1.201369  -0.121970 -0.043875   
2       232022460    1 -0.172107 -0.184668 -1.201369  -0.121970 -0.043875   
3       232438397    1  0.163054  1.963790 -0.646376  12.409294 -0.063725   
4       230460314    1  1.011523 -0.081127 -1.201369   1.153668  0.333276   
...           ...  ...       ...       ...       ...        ...       ...   
203764  173077460   49 -0.145771 -0.163752  0.463609  -0.121970 -0.043875   
203765  158577750   49 -0.165920 -0.123607  1.018602  -0.121970 -0.043875   
203766  158375402   49 -0.172014 -0.078182  1.018602   0.028105 -0.043875   
203767  158654197   49 -0.172842 -0.176622  1.018602  -0.121970 -0.043875   
203768  157597225   49 -0.012037 -0.132276  0.463609  -0.121970 -0.043875   

             7          8         9    ...       157       158       159  \

In [20]:

df_classes = df_classes.loc[df_classes.iloc[:, 1] != 'unknown']

# Ora df contiene solo le righe in cui il valore nella seconda colonna non è 'unknown'
print(df_classes)

nodi_da_mantenere = set(df_classes['txId'])

df_edgelist = df_edgelist[df_edgelist['txId1'].isin(nodi_da_mantenere) & df_edgelist['txId2'].isin(nodi_da_mantenere)]


             txId class
3       232438397     2
9       232029206     2
10      232344069     2
11       27553029     2
16        3881097     2
...           ...   ...
203752   80329479     2
203754  158406298     2
203759  158375075     1
203763  147478192     2
203766  158375402     1

[46564 rows x 2 columns]
            txId1      txId2
5       232344069   27553029
8         3881097  232457116
15      232051089  232470704
26      230473487    7089694
33      231182296   14660781
...           ...        ...
234338  194020062   47521535
234340  158574502  109383451
234344  158594124  157631640
234347  157631640   21644119
234350  158365409  157930723

[36624 rows x 2 columns]


Rete 

In [21]:
import networkx as nx

import matplotlib.pyplot as plt  # Solo se desideri visualizzare il grafico

print("sono qui")

# Crea un grafo non diretto da edgelist
G = nx.from_pandas_edgelist(df_edgelist, 'txId1', 'txId2')

print("sono qui")
print(G)


# Disegna il grafo
pos = nx.spring_layout(G)  # Posiziona i nodi in modo che siano ben distribuiti

# Disegna i nodi senza etichette
nx.draw(G, pos, with_labels=False, node_size=20, node_color='red')

# Disegna gli archi
nx.draw_networkx_edges(G, pos, width=0.02, edge_color='gray')

# Mostra il grafico
plt.axis('off')  # Nasconde gli assi
plt.show()

sono qui
sono qui
Graph with 203769 nodes and 234355 edges


KeyboardInterrupt: 